# Artificial Neural Network

### Importing libraries

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
print(tf.__version__)

2.13.0


## Data Preprocessing

### Importing dataset

In [6]:
dataset = pd.read_csv('./Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [7]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Encoding categorical data

#### Label encoding gender column

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [15]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


#### One Hot Encoding of Country

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [17]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting dataset into Training set and Test set

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

### Feature scaling

In [19]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Build the ANN

### Initialize ANN

In [20]:
ann = tf.keras.models.Sequential()

### Addin input layer and first hidden layer

In [21]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding a second hidden layer

In [22]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [23]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the ANN

### Compiling the ANN

In [24]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the Training set

In [25]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 702us/step - loss: 0.5894 - accuracy: 0.7240
Epoch 2/100
250/250 [==============================] - 0s 666us/step - loss: 0.4677 - accuracy: 0.8004
Epoch 3/100
250/250 [==============================] - 0s 662us/step - loss: 0.4387 - accuracy: 0.8004
Epoch 4/100
250/250 [==============================] - 0s 698us/step - loss: 0.4249 - accuracy: 0.8090
Epoch 5/100
250/250 [==============================] - 0s 661us/step - loss: 0.4153 - accuracy: 0.8216
Epoch 6/100
250/250 [==============================] - 0s 652us/step - loss: 0.4059 - accuracy: 0.8260
Epoch 7/100
250/250 [==============================] - 0s 678us/step - loss: 0.3960 - accuracy: 0.8338
Epoch 8/100
250/250 [==============================] - 0s 684us/step - loss: 0.3851 - accuracy: 0.8401
Epoch 9/100
250/250 [==============================] - 0s 690us/step - loss: 0.3740 - accuracy: 0.8476
Epoch 10/100
250/250 [==============================] - 0s 659us/step - l

## Making predictions and evaluating the model

### Predicting the result of a single observation

In [26]:
probability = ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))
print(probability)

1/1 [==============================] - 0s 69ms/step
[[0.02658909]]


### Predicting the Test set Results

In [27]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 581us/step
[[0 0]
 [0 0]
 [1 1]
 ...
 [0 0]
 [0 0]
 [0 1]]


### Confusion Matrix

In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1492   68]
 [ 227  213]]


0.8525